In [ ]:
%reset
%load_ext autoreload
%autoreload 2

import Sampling
import UsefulFunctions as fun
import DeformationModules as df
import Models
import Hamiltonian

import numpy as np
import matplotlib.pyplot as plt
import torch

torch.set_default_tensor_type(torch.DoubleTensor)

In [ ]:
source = Sampling.sampleFromGreyscale("data/density_a.png")
target = Sampling.sampleFromGreyscale("data/density_b.png")
source[0].requires_grad_()

In [ ]:
fun.plotTensorScatter(source, 0.4)
fun.plotTensorScatter(target, 0.4)
plt.show()

In [ ]:
points = torch.cat((source[0], target[0]), 0).detach().numpy()
minx, miny, maxx, maxy = np.min(points[:, 0]), np.min(points[:, 1]), np.max(points[:, 0]), np.max(points[:, 1])

In [ ]:
nbGD = 500
GD = torch.rand(nbGD, 2)
GD[:, 0] = (maxx - minx) * GD[:, 0] + minx
GD[:, 1] = (maxy - miny) * GD[:, 1] + miny
GD = GD.view(-1)

In [ ]:
nbGD = 1000
x, y = torch.meshgrid([torch.arange(minx, maxx, step=(maxx-minx)/np.sqrt(nbGD)), torch.arange(miny, maxy, step=(maxy-miny)/np.sqrt(nbGD))])
GD = fun.grid2vec(x, y)
GD = GD.contiguous().view(-1)


In [ ]:
fun.plotTensorScatter(source)
fun.plotTensorScatter(target)
plt.plot(GD.view(-1, 2).detach()[:, 1].numpy(), GD.view(-1, 2).detach()[:, 0].numpy(), '.')
plt.show()

In [ ]:
myModel = Models.ModelTranslationModuleRegistration(0.05, 2, source, GD)

costs = myModel.fit(target, 100)

In [ ]:
out = myModel()
outGD, outMOM = myModel.getVars()
%matplotlib qt5
fun.plotTensorScatter(target, alpha=0.4)
fun.plotTensorScatter(out, alpha=0.4)
plt.plot(GD.view(-1, 2).detach()[:, 1].numpy(), GD.view(-1, 2).detach()[:, 0].numpy(), '.')
plt.quiver(y, x, outMOM.view(-1, 2).detach()[:, 1].numpy(), outMOM.view(-1, 2).detach()[:, 0].numpy())
plt.show()

In [ ]:
plt.plot(range(0, len(costs)), costs)
plt.show()